## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 9/13/2024)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

%load_ext autoreload
%autoreload 2